In [2]:
import subprocess

import pandas as pd
from pathlib import Path


In [3]:

reference_pdfs_folder_path = Path(r"/home/vedant/Downloads/reference-pdfs-v1")
from video2pdf.utils.evaluation import get_pdf_files, build_pdf_files_df, clean_df

# ---- Get pdf files from folder
reference_pdf_files = get_pdf_files(reference_pdfs_folder_path)

# ---- Build df for the pdf files
# ---- Columns would be `internal_id`, `pdf_path`
reference_pdf_files_df = build_pdf_files_df(reference_pdf_files)

# ---- Clean df; drop duplicate pdfs for same internal_id
reference_pdf_files_df = clean_df(reference_pdf_files_df)


In [4]:

input_file = "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/experiments/27/ref_timestamps_with_internal_id.csv"


In [5]:
df = pd.read_csv(input_file)
df.shape

(120, 4)

In [6]:
df = df[df["internal_id"].isin(reference_pdf_files_df["internal_id"])]
df.shape


(91, 4)

In [7]:
df.columns

Index(['internal_id', 'video_id', 'video_name', 'timestamps'], dtype='object')

In [12]:
from video2pdf.utils.constants import BASE_DIR

picke_inputs = list(Path(BASE_DIR).glob("*_python_object"))

In [13]:
from typing import List


def get_pickle_input(internal_id, picke_inputs: List[Path]):
    picke_input = [i.name for i in picke_inputs if internal_id in i.name]
    return picke_input[0]

get_pickle_input("adwxvg", picke_inputs)

'adwxvg_oua_python_object'

In [16]:
for internal_id,_,_,timestamps in df.values:
    timestamps = eval(timestamps)
    # timestamps = str(timestamps)
    timestamps = list(map(str, timestamps))
    timestamps = f"{(','.join(timestamps))}"
    if len(internal_id) != 6:
        continue
    try:
        pickle_input = get_pickle_input(internal_id, picke_inputs)
        args = [
            "python", "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/main.py",
            "--input=pickle",
            f"--dir={pickle_input}",
            "--ocr=tesseract",
            "--extraction=timestamps",
            f'--timestamps',
            timestamps,
            '--ocr_approval=reject_all'
        ]
        subprocess.run(
            args
        )
    except:
        print("Exception occurred for: ", internal_id)

Traceback (most recent call last):
  File "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/main.py", line 119, in <module>
    main()
  File "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/main.py", line 109, in main
    directory = input_strategy.process()
                ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/input_strategy/base.py", line 75, in process
    output_dir = self.save_frames(extracted_frames)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/input_strategy/base.py", line 145, in save_frames
    Helper.save_extracted_frames(
  File "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/utils/helper.py", line 67, in save_extracted_frames
    cv2.imwrite(frame_output_path, frame)
cv2.error: OpenCV(4.10.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:798: error: (-215:Assertion failed) !_

In [30]:

    dir = "/home/vedant/Desktop/glimpsify/most_info_frame_extractor/video2pdf/data"
